In [ ]:
# ------------------------------------------------------
# IMPORT DES LIBRAIRIES
# ------------------------------------------------------

import pandas as pd               # pour lire et manipuler les données tabulaires (features.csv)
import numpy as np                # pour manipuler des tableaux numériques
from sklearn.model_selection import train_test_split, StratifiedKFold  # split dataset + CV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler  # normalisation
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score  # métriques d'évaluation
from xgboost import XGBClassifier  # modèle XGBoost
from imblearn.over_sampling import SMOTE  # pour gérer le déséquilibre
import warnings
warnings.filterwarnings("ignore")  # pour éviter les warnings inutiles

# ------------------------------------------------------
# CONFIGURATION GENERALE (à modifier selon ton dataset)
# ------------------------------------------------------

FEATURES_PATH = "data/features.csv"   # ⚠️ chemin du fichier CSV contenant tes features
TARGET_COLUMN = "label"               # ⚠️ nom de la colonne contenant la classe cible
SEED = 42                             # graine aléatoire pour reproductibilité

# ------------------------------------------------------
# CHARGEMENT DU DATASET
# ------------------------------------------------------

print("📥 Chargement du dataset...")
df = pd.read_csv(FEATURES_PATH)       # lecture du CSV dans un DataFrame pandas

# Séparation features (X) et labels (y)
X = df.drop(columns=[TARGET_COLUMN]).values  # toutes les colonnes sauf la cible
y = df[TARGET_COLUMN].values                 # la colonne cible (labels)

print(f"✅ Dataset chargé : {X.shape[0]} échantillons et {X.shape[1]} features.")

# ------------------------------------------------------
# DEFINIR LES SCALERS A TESTER
# ------------------------------------------------------
# Chaque scaler applique une méthode de normalisation différente
scalers = {
    "StandardScaler": StandardScaler(),  # moyenne = 0, variance = 1
    "MinMaxScaler": MinMaxScaler(),      # toutes les valeurs entre 0 et 1
    "RobustScaler": RobustScaler()       # moins sensible aux valeurs extrêmes
}

# ------------------------------------------------------
# DEFINIR LES CONFIGURATIONS XGBOOST A TESTER
# ------------------------------------------------------
configs = {
    "Baseline": {"n_estimators": 200, "learning_rate": 0.1, "max_depth": 6},
    "Deep Trees": {"n_estimators": 300, "learning_rate": 0.05, "max_depth": 10},
    "Shallow Trees": {"n_estimators": 500, "learning_rate": 0.01, "max_depth": 3}
}

# ------------------------------------------------------
# FONCTION D'EVALUATION DU MODELE
# ------------------------------------------------------
def evaluate(y_true, y_pred, dataset_name="Test"):
    """
    Calcule et affiche Accuracy, F1-score et ROC-AUC (si applicable).
    """
    # Accuracy = proportion de prédictions correctes
    acc = accuracy_score(y_true, y_pred)

    # F1-score pondéré = équilibre entre précision et rappel
    f1 = f1_score(y_true, y_pred, average="weighted")

    # ROC-AUC = aire sous la courbe ROC (⚠️ calculable seulement en binaire)
    try:
        auc = roc_auc_score(pd.get_dummies(y_true), pd.get_dummies(y_pred), average="weighted")
    except:
        auc = None

    # Affichage des résultats
    print(f"📊 {dataset_name} | Accuracy = {acc:.4f}, F1 = {f1:.4f}", end="")
    if auc is not None:
        print(f", ROC-AUC = {auc:.4f}")
    else:
        print(" (ROC-AUC non calculable - multi-classes détecté)")

    return acc, f1, auc

# ------------------------------------------------------
# BOUCLE PRINCIPALE DE TEST
# ------------------------------------------------------

results = []  # liste où on stocke tous les résultats

# Boucle sur les SCALERS
for scaler_name, scaler in scalers.items():
    # Application du scaler sur les features
    X_scaled = scaler.fit_transform(X)

    # Split du dataset en TRAIN (80%) et TEST (20%)
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y, test_size=0.2, stratify=y, random_state=SEED
    )

    # Boucle sur les CONFIGS XGBoost
    for config_name, params in configs.items():
        print(f"\n🚀 Test avec {scaler_name} + Config = {config_name}")

        # Création du modèle avec les paramètres choisis
        model = XGBClassifier(
            use_label_encoder=False,
            eval_metric="logloss",
            random_state=SEED,
            **params
        )

        # ------------------------------------------------------
        # VARIANTE 3 : Validation croisée (K-Fold)
        # ------------------------------------------------------
        kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
        f1_scores = []  # stocke les scores F1 de chaque fold

        for fold, (train_idx, val_idx) in enumerate(kf.split(X_train, y_train)):
            # Séparation du TRAIN et VALIDATION pour ce fold
            X_tr, X_val = X_train[train_idx], X_train[val_idx]
            y_tr, y_val = y_train[train_idx], y_train[val_idx]

            # ------------------------------------------------------
            # VARIANTE 4 : Gestion déséquilibre avec SMOTE
            # ------------------------------------------------------
            smote = SMOTE(random_state=SEED)  # création de l'objet SMOTE
            X_tr_bal, y_tr_bal = smote.fit_resample(X_tr, y_tr)  # oversampling des classes rares

            # Entraînement du modèle sur données rééquilibrées
            model.fit(X_tr_bal, y_tr_bal)

            # Prédictions sur la validation
            y_val_pred = model.predict(X_val)

            # Score F1 pour ce fold
            f1_fold = f1_score(y_val, y_val_pred, average="weighted")
            f1_scores.append(f1_fold)

            print(f"   Fold {fold+1} : F1 = {f1_fold:.4f}")

        # Moyenne des scores de validation croisée
        print(f"   ➡️ Moyenne F1 CV = {np.mean(f1_scores):.4f} ± {np.std(f1_scores):.4f}")

        # ------------------------------------------------------
        # 18. EVALUATION FINALE SUR LE TEST SET
        # ------------------------------------------------------
        # Réentraînement du modèle sur tout le TRAIN rééquilibré
        smote = SMOTE(random_state=SEED)
        X_train_bal, y_train_bal = smote.fit_resample(X_train, y_train)
        model.fit(X_train_bal, y_train_bal)

        # Prédictions sur le test set
        y_test_pred = model.predict(X_test)

        # Evaluation complète
        acc, f1, auc = evaluate(y_test, y_test_pred, dataset_name="Test final")

        # Stockage du résultat dans la liste
        results.append({
            "Scaler": scaler_name,
            "Config": config_name,
            "CV_F1_mean": np.mean(f1_scores),
            "CV_F1_std": np.std(f1_scores),
            "Test_Accuracy": acc,
            "Test_F1": f1,
            "Test_AUC": auc
        })

# ------------------------------------------------------
# AFFICHAGE DU TABLEAU FINAL DES RESULTATS
# ------------------------------------------------------
results_df = pd.DataFrame(results)
print("\n📊 Tableau comparatif des résultats (toutes variantes) :")
print(results_df)
